In [19]:
from google.colab import drive
import sys
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir rutas base y repo
BASE_PATH = '/content/drive/MyDrive/Tesis'
REPO_NAME = 'Tesis_NN'
REPO_PATH = os.path.join(BASE_PATH, REPO_NAME)

# Agregar repo al path de Python para imports
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)

# Cambiar directorio actual al repo (para comandos git)
os.chdir(REPO_PATH)

print(f"Drive montado. Carpeta repo seteada en: {REPO_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive montado. Carpeta repo seteada en: /content/drive/MyDrive/Tesis/Tesis_NN


In [20]:
from google.colab import files
%cd /content/drive/MyDrive/Tesis/Tesis_NN
!pip install -r requirements.txt

/content
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.14-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached executing-2.2.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached fonttools-4.58.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (106 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipython-9.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached ipython_pygments_lexers-1.1.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8

In [21]:
!pip install webdataset

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 📦 Standard Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import os
import sys
import time
import csv
import json
from pathlib import Path

# ───────────────────────────────────────────────────────────────────────────────
# 📚 Scientific & Data Libraries
# ───────────────────────────────────────────────────────────────────────────────
import numpy as np
import matplotlib.pyplot as plt

# ───────────────────────────────────────────────────────────────────────────────
# 🌍 Third-Party Library Imports (PyTorch)
# ───────────────────────────────────────────────────────────────────────────────
import torch
import torch.optim as optim

# ───────────────────────────────────────────────────────────────────────────────
# 🧩 Custom Project Modules
# ───────────────────────────────────────────────────────────────────────────────
# Add custom project folder to system path to enable local module imports
if os.name == "posix":
    sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
else:
    sys.path.append('C:/Users/nnobi/Desktop/FIUBA/Tesis/Project')

from project_package.utils import train_common_routines as tcr
from project_package.conv_net.ConvNet_model import SRCNN_small
from project_package.dataset_manager.webdataset_dataset import PtWebDataset

# ───────────────────────────────────────────────────────────────────────────────
# 🔧 Configuration
# ───────────────────────────────────────────────────────────────────────────────
model_selection = 'SRCNN_small'
epochs = 200
lr = 1e-5
batch_size = 32

# ───────────────────────────────────────────────────────────────────────────────
# 📁 Paths Setup
# ───────────────────────────────────────────────────────────────────────────────
script_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(script_dir, '..'))

dataset = 'dataset_test' # Select Dataset
dataset_folder = os.path.join(project_dir, 'datasets', dataset)
metadata_path = os.path.join(dataset_folder, 'metadata.json')

with open(metadata_path, "r") as f:
    metadata = json.load(f)

train_samples = metadata["splits"]["train"]["num_samples"]
val_samples = metadata["splits"]["val"]["num_samples"]
test_samples = metadata["splits"]["test"]["num_samples"]

# Results folder and files
results_folder = os.path.join(project_dir, 'results', model_selection)
os.makedirs(results_folder, exist_ok=True)

loss_png_file = os.path.join(results_folder, f"loss_lr={lr}_batch_size={batch_size}_model={model_selection}.png")
psnr_png_file = os.path.join(results_folder, f"psnr_lr={lr}_batch_size={batch_size}_model={model_selection}.png")
final_model_pth_file = os.path.join(results_folder, f"model_lr={lr}_batch_size={batch_size}_model={model_selection}.pth")
file_training_losses = os.path.join(results_folder, f"training_losses_lr={lr}_batch_size={batch_size}_model={model_selection}.csv")


In [ ]:

# ───────────────────────────────────────────────────────────────────────────────
# 🚀 Training Pipeline
# ───────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":

    # 🧠 Model Initialization
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    torch.backends.cudnn.benchmark = True # Enables cuDNN auto-tuner to find the best algorithm for fixed input sizes (improves performance)

    model = SRCNN_small().to(device)
    print("The model:")
    print(model)

    model.count_parameters()
    print(f"Total Parameters: {model.total_params:,}")
    print(f"Trainable Parameters: {model.trainable_params:,}")

    model = tcr.multi_GPU_training(model)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 📊 Dataset and DataLoaders
    # Tune num_workers for dataloaders and buffersize
    dataset_train = PtWebDataset(os.path.join(dataset_folder,'train-*.tar'), length=train_samples, batch_size=batch_size, shuffle_buffer=5 * batch_size)
    dataset_val = PtWebDataset(os.path.join(dataset_folder,'val-*.tar'), length=val_samples, batch_size=batch_size, shuffle_buffer=5 * batch_size)
    dataset_test = PtWebDataset(os.path.join(dataset_folder,'test.tar'), length=test_samples, batch_size=batch_size, shuffle_buffer=5 * batch_size)

    dataloader_train = dataset_train.get_dataloader(num_workers=1)
    dataloader_val = dataset_val.get_dataloader(num_workers=1)
    dataloader_test = dataset_test.get_dataloader(num_workers=1)

    # 🏋️ Training Loop
    train_loss, val_loss = [], []
    train_psnr, val_psnr = [], []

    start = time.time()
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1} of {epochs}")

        train_epoch_loss, train_epoch_psnr = tcr.train(model, dataloader_train, optimizer, tcr.compute_loss_MSE, device, train_samples)
        val_epoch_loss, val_epoch_psnr = tcr.validate(model, dataloader_val, epoch, tcr.compute_loss_MSE, device, val_samples)

        print(f"Train PSNR: {train_epoch_psnr:.3f}")
        print(f"Val PSNR: {val_epoch_psnr:.3f}")

        train_loss.append(train_epoch_loss)
        train_psnr.append(train_epoch_psnr)
        val_loss.append(val_epoch_loss)
        val_psnr.append(val_epoch_psnr)

        if epoch % 5 == 0:
            checkpoint_path = os.path.join(results_folder, f"checkpoint_epoch_{epoch}_lr={lr}_batch_size={batch_size}_model={model_selection}.pth")
            tcr.save_checkpoint(epoch, model, optimizer, train_loss, filename=checkpoint_path)

        with open(file_training_losses, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([train_epoch_loss, train_epoch_psnr, val_epoch_loss, val_epoch_psnr])

    end = time.time()
    print(f"\n✅ Finished training in: {(end - start) / 60:.2f} minutes")

    # 📈 Loss Plot
    plt.figure(figsize=(10, 7))
    plt.plot(10 * np.log10(train_loss), color='orange', label='Train Loss')
    plt.plot(10 * np.log10(val_loss), color='red', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (dB)')
    plt.legend()
    if os.path.exists(loss_png_file):
        os.remove(loss_png_file)
    plt.savefig(loss_png_file)
    plt.show()

    # 📈 PSNR Plot
    plt.figure(figsize=(10, 7))
    plt.plot(train_psnr, color='green', label='Train PSNR (dB)')
    plt.plot(val_psnr, color='blue', label='Validation PSNR (dB)')
    plt.xlabel('Epochs')
    plt.ylabel('PSNR (dB)')
    plt.legend()
    if os.path.exists(psnr_png_file):
        os.remove(psnr_png_file)
    plt.savefig(psnr_png_file)
    plt.show()

    # 💾 Save Final Model
    print('\n💾 Saving model...')
    if os.path.exists(final_model_pth_file):
        os.remove(final_model_pth_file)
    model = model.module if hasattr(model, "module") else model
    torch.save(model.state_dict(), final_model_pth_file)
